In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp

class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1], 
                     use_bias=True, 
                     param_dtype=np.complex128, 
                     kernel_init=nn.initializers.normal(stddev=0.01), 
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x   

In [24]:
global trained_params_list 
global parameters_list     

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [36]:
def info(e):
    head   = list(e.keys())[0]
    body   = list(e[head].keys())
    bias   = e[head][body[0]]
    kernel = e[head][body[1]]
    return  head, body, list(bias), list(kernel)
def real(c):
    return float(np.real(c))  
def img(c):
    return float(np.imag(c))    
def r_i(c):
    return real(c),img(c)   

In [5]:
# Define a função de perda (loss function) para o treinamento
def loss(params, structure_factor):
    output = model.apply({'params': params}, structure_factor)
    return jnp.mean(output)

In [27]:
def run(it):
    print("------------",it,"-------------")
    J = [1, 0.2];L = 8;edge_colors = []
    for i in range(L):
        edge_colors.append([i, (i+1)%L, 1])
        edge_colors.append([i, (i+2)%L, 2])
    g = nk.graph.Graph(edges=edge_colors)
    sigmaz = [[1, 0], [0, -1]];  mszsz = (np.kron(sigmaz, sigmaz))
    exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])
    bond_operator = [
        (J[0] * mszsz).tolist(),    (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),(J[1] * exchange).tolist(),
        ]
    bond_color = [1, 2, 1, 2];   sigmaz = [[1, 0], [0, -1]]
    mszsz = (np.kron(sigmaz, sigmaz));
    exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])
    bond_operator = [
        (J[0] * mszsz).tolist(),    (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),(J[1] * exchange).tolist(),
    ]
    bond_color = [1, 2, 1, 2]
    hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)
    op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)
    model = FFNN()
    sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, d_max = 2)
    vs = nk.vqs.MCState(sa, model, n_samples=1008)
    vs_i_parameters = vs.parameters.copy()    
    opt = nk.optimizer.Sgd(learning_rate=0.01)
    sr = nk.optimizer.SR(diag_shift=0.01)
    gs = nk.VMC(hamiltonian=op, optimizer=opt, variational_state=vs, preconditioner=sr)
    sf = []
    sites = []
    structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
    for i in range(0, L):
        for j in range(0, L):
            structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L
    trained_params_list = []
    parameters_list     = []
    
    # Função de callback para salvar os parâmetros durante o treinamento
    def save_params(step, params, energy):
        trained_params_list.append(params.copy())
        parameters_list.append(energy.state.parameters.copy())
        return True
    gs.run(out='test', n_iter=it, obs={'Structure Factor': structure_factor}, callback=save_params)
    print("------------  -------------")
    return vs_i_parameters, vs.parameters, parameters_list

In [28]:
vs_i_parameters, parameters, parameters_list  = run(100)

------------ 100 -------------


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 17.49it/s, Energy=-13.4056-0.0012j ± 0.0021 [σ²=0.0046, R̂=1.0105]]

------------  -------------
100


In [30]:
vs_i_parameters['Dense_0']['bias']

Array([ 0.0107735 -5.90024997e-03j, -0.00348329+1.86208617e-05j,
        0.00278065+2.49753137e-03j,  0.0041145 +1.09895453e-02j,
        0.01533332-3.55332764e-03j,  0.00269199+3.48258846e-03j,
        0.00079142-2.90021583e-03j,  0.00165894+4.83000123e-03j,
       -0.00613418-2.58584781e-03j, -0.0161698 -4.91889383e-03j,
        0.00044501+5.16526800e-03j,  0.00473567-4.14902071e-03j,
       -0.00297952-8.57810607e-03j, -0.00555106-4.15510558e-03j,
        0.00732408+2.42429511e-03j,  0.00133757+1.94564498e-03j],      dtype=complex128)

In [32]:
parameters['Dense_0']['bias']

Array([ 0.00629866-0.00618058j,  0.00168721+0.0002418j ,
        0.00241711+0.00417768j, -0.00246286+0.00507068j,
        0.01121054-0.00413867j,  0.0001007 +0.00309828j,
        0.00048257-0.00094045j,  0.00244095+0.00308909j,
        0.00202159-0.0019732j , -0.00465634-0.0030279j ,
       -0.00026279+0.00235873j,  0.00175976-0.00324178j,
       -0.00466339-0.00375461j, -0.00380667-0.00364623j,
        0.00792056+0.00174386j,  0.00202289+0.00268519j],      dtype=complex128)

In [33]:
parameters_list[-1]['Dense_0']['bias']

Array([ 6.32964393e-03-0.00617504j,  1.69247856e-03+0.00022558j,
        2.43571053e-03+0.00417078j, -2.51141180e-03+0.00507832j,
        1.11879965e-02-0.00415628j,  7.53256747e-05+0.00311069j,
        4.13275658e-04-0.00094683j,  2.50464239e-03+0.00315494j,
        1.97785607e-03-0.00205708j, -4.73351279e-03-0.00309966j,
       -3.73175974e-04+0.0024452j ,  1.75017302e-03-0.00326143j,
       -4.63040972e-03-0.00376002j, -3.80728103e-03-0.00371172j,
        7.94004932e-03+0.00166277j,  2.00890534e-03+0.00273026j],      dtype=complex128)

In [37]:
head_i, body_i, bias_i, kernel_i   = info(parameters); 

In [39]:
head_i

'Dense_0'

In [42]:
it = 0; bia_i_list  = [];bia_f_list = [] ; kernel_i_list = []; kernel_f_list = []
lm =[0,100]; step = 10
for i in range(lm[0],lm[1],step):
    vs_i_parameters, parameters, parameters_list  = run(i)
    head_i, body_i, bias_i, kernel_i   = info(vs_i_parameters); 
    head_f, body_f, bias_f, kernel_f  = info(parameters)
    bia_i_list.append(bias_i)
    bia_f_list.append(bias_f)
    kernel_i_list.append(kernel_i)
    kernel_f_list.append(kernel_f)    

------------ 0 -------------


0it [00:00, ?it/s]


------------  -------------
0
------------ 10 -------------


100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 20.86it/s, Energy=-12.391-0.000j ± 0.099 [σ²=9.838, R̂=1.0065]]


------------  -------------
10
------------ 20 -------------


...

------------  -------------
70
------------ 80 -------------


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:04<00:00, 17.27it/s, Energy=-13.4013+0.0003j ± 0.0018 [σ²=0.0034, R̂=1.0017]]


------------  -------------
80
------------ 90 -------------


100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:05<00:00, 16.76it/s, Energy=-13.4047-0.0035j ± 0.0026 [σ²=0.0071, R̂=1.0098]]

------------  -------------
90


In [ ]:
head_i, body_i, bias_i, kernel_i   = info(parameters); 

head_g, body_g, bias_g, kernel_g   = info(bia_i_list[-1])
head_i, body_i, bias_i, kernel_i   = info(vs_i)

In [ ]:
for l in kernel_i:
        for m in l:
            print(r_i(m)) 
            break;

In [ ]:
for l in kernel_g:
        for m in l:
            print(r_i(m)) 
            break;

In [ ]:
for b in bias_g:
    print(r_i(b))
    break;

In [ ]:
for b in bias_i:
    print(r_i(b))
    break;

In [ ]:
for b in bias_i:
    print(r_i(b))
    break;

In [ ]:
g

In [ ]:
vs_i

In [ ]:
for bia in bia_i_list:
    for b in bia:
        print(r_i(b))

In [ ]:
for bia in bia_f_list:
    for b in bia:
        print(r_i(b))

In [ ]:
for kernel in kernel_i_list:
    for l in kernel:
        for m in l:
            print(r_i(m))    

In [ ]:
for kernel in kernel_f_list:
    for l in kernel:
        for m in l:
            print(r_i(m))      